In [1]:
import config
import logging

from IPython import display
from src.TPL import *
import src.preprocessing as pre


import numpy as np
import pandas as pd
import time 
import gensim 
from gensim import corpora, models, similarities
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import ftn
import pickle

%matplotlib inline
%reload_ext autoreload
%autoreload 2
logging.basicConfig(level=logging.INFO)

In [2]:
config.ds_medium

{'startTimestamp': '2020-12-01T00:00:00.000',
 'stopTimestamp': '2021-01-02T00:00:00.000',
 'dataset_dir': '../../../data/raw',
 'processed_dir': '../../../data/processed'}

In [10]:
class NumbersAndTelescopes(pre.Numbers, pre.Telescopes):
    pass

colorize1 = pre.Numbers()
colorize2 = NumbersAndTelescopes()

### CountVectorizer() para sentences:

El problema es el siguiente, si bien el modelo LDA_sklt está funcionando y arroja resultados potencialmente útiles, le está faltando el respeto (mejor dicho, le estoy faltando el respeto) a la premisa principal para trabajar el dataset: cada evento es un token, debido a que CountVecotirezer() crea los tokens palabra por palabra. Veamos un ejemplo:

In [44]:
#Importamos la herramienta:
from sklearn.feature_extraction.text import CountVectorizer

#Corpus para ejemplificar:
corp = [
    'This is the first document.',
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',]
#Clase del CountVectorizer()
vectorizer = CountVectorizer()
#Aplicamos el CV
X = vectorizer.fit_transform(corp)
#Mostramos los nombres:
vectorizer.get_feature_names_out()

array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

In [4]:
#Mostramos un array que contiene la frecuencia de aparición de todos los tokens del corpus para cada oración
X.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

Como se puede ver, divide a cada oración palabra por palabra y luego tokeniza. Lo que queremos entonces es evitar que haga esta división y así trabajar event-token. Para ello, reemplacemos cada espacio dentro de la oración por un "_" y veamos en qué resulta

In [5]:
corp_sin_espacios = list(map(lambda x: x.replace(' ','_'), corp))
corp_sin_espacios

['This_is_the_first_document.',
 'This_is_the_first_document.',
 'This_document_is_the_second_document.',
 'And_this_is_the_third_one.',
 'Is_this_the_first_document?']

In [6]:
Y = vectorizer.fit_transform(corp_sin_espacios)
#Mostramos los nombres:
vectorizer.get_feature_names_out()

array(['and_this_is_the_third_one', 'is_this_the_first_document',
       'this_document_is_the_second_document',
       'this_is_the_first_document'], dtype=object)

In [7]:
Y.toarray()

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]], dtype=int64)

Mucho mejor que antes, ahora que contamos con una hipótesis para solucionar el problema, probémosla para nuestro dataset. 

In [8]:
pionier1=TplPionier(config=config.ds_medium)
# Some rows are small (<5 sec) or unbounded (>30m), this criteria will restrict to only the good ones
pionier1.index = pionier1.index[pionier1.index["SECONDS"] > 5]
pionier1.index = pionier1.index[pionier1.index["SECONDS"] < 1800]
pionier1.index

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,estimated (s),real (s),diff (s),SECONDS
0,2020-12-01 06:58:21.518000+00:00,2020-12-01 07:01:26.742000+00:00,False,PIONIER,bob_ins,PIONIER_gen_tec_scan,True,False,5,2,3,185.0
4,2020-12-01 07:10:51.269000+00:00,2020-12-01 07:20:26.003000+00:00,False,PIONIER,bob_ins,PIONIER_obs_calibrator,False,False,308,935,387,574.0
5,2020-12-01 07:20:26.022000+00:00,2020-12-01 07:24:22.424000+00:00,False,PIONIER,bob_ins,PIONIER_gen_cal_kappa,False,False,125,356,151,236.0
6,2020-12-01 07:24:22.441000+00:00,2020-12-01 07:26:17.805000+00:00,False,PIONIER,bob_ins,PIONIER_gen_cal_dark,False,False,46,155,69,115.0
7,2020-12-01 07:26:38.306000+00:00,2020-12-01 07:29:55.510000+00:00,False,PIONIER,bob_ins,PIONIER_acq,True,False,400,317,43,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2839,2021-01-01 19:09:48.738000+00:00,2021-01-01 19:10:13.501000+00:00,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,30,25,5,24.0
2840,2021-01-01 19:10:13.517000+00:00,2021-01-01 19:11:33.481000+00:00,False,PIONIER,bob_ins,PIONIER_gen_tec_niobate,False,False,100,120,20,79.0
2841,2021-01-01 19:11:33.498000+00:00,2021-01-01 19:11:58.376000+00:00,False,PIONIER,bob_ins,PIONIER_gen_tec_setup,False,False,30,25,5,24.0
2842,2021-01-01 19:11:58.394000+00:00,2021-01-01 19:12:33.041000+00:00,False,PIONIER,bob_ins,PIONIER_gen_tec_standby,False,False,100,35,25,34.0


In [11]:
#Trace con los eventos colorizados que queremos convertir en tokens:
primerafila=[]
for a in pionier1.load_trace(0)["event"]:
    primerafila.append(colorize2.color(a))
#Mostramos la trace
primerafila

['Started at {} (underlined)',
 "INS MODE = 'OBSERV-H'",
 "ISS IAS HMAG = '{}'",
 "SEQ OPL BEAM3 = '{}'",
 "SEQ OPL BEAM4 = '{}'",
 "INS DISP NAME = 'GRISM'",
 "TEL COU ALPHA = '{}'",
 "ISS BASELINE = 'large medium'",
 "TEL COU EPOCH = '{}'",
 "SEQ DL PRESET = 'T'",
 "ISS VLTITYPE = 'snapshot'",
 "SEQ OPL BEAM1 = '{}'",
 "SEQ OPL BEAM2 = '{}'",
 "SEQ PRESET = 'T'",
 "SEQ PUPIL = 'F'",
 "TEL COU DELTA = '{}'",
 "AOS AO MODE = 'FULL_AO'",
 "TEL COU PMA = '{}'",
 "TEL TARG ADDVELDELTA = '{}'",
 "TEL TARG EQUINOX = '{}'",
 "TEL TARG PMA = '{}'",
 "TEL COU PMD = '{}'",
 "TEL TARG ADDVELALPHA = '{}'",
 "TEL COU MAG = '{}'",
 "TEL COU EQUINOX = '{}'",
 "TEL TARG ALPHA = '{}'",
 "TEL TARG PMD = '{}E-{}'",
 "TEL DEL REF = '{}'",
 "TEL COU GSTYPE = 'ADAPT_OPT'",
 "TEL TARG DELTA = '{}'",
 "TEL DEL MODE = 'AUTO'",
 "TEL COU GSSOURCE = 'SCIENCE'",
 "TEL DELREF POS = '{}'",
 "TEL TARG EPOCH = '{}'",
 'Send command CHKMOON {},{},{} to process issifControl env. wvgvlti',
 'Waiting (timeout {}) for pe

In [12]:
#Trace con los espacios cambiados por "_" (SE= sin espacios)
primerafila_SE=list(map(lambda x: x.replace(' ','_'), primerafila))
primerafila_SE

['Started_at_{}_(underlined)',
 "INS_MODE_=_'OBSERV-H'",
 "ISS_IAS_HMAG_=_'{}'",
 "SEQ_OPL_BEAM3_=_'{}'",
 "SEQ_OPL_BEAM4_=_'{}'",
 "INS_DISP_NAME_=_'GRISM'",
 "TEL_COU_ALPHA_=_'{}'",
 "ISS_BASELINE_=_'large_medium'",
 "TEL_COU_EPOCH_=_'{}'",
 "SEQ_DL_PRESET_=_'T'",
 "ISS_VLTITYPE_=_'snapshot'",
 "SEQ_OPL_BEAM1_=_'{}'",
 "SEQ_OPL_BEAM2_=_'{}'",
 "SEQ_PRESET_=_'T'",
 "SEQ_PUPIL_=_'F'",
 "TEL_COU_DELTA_=_'{}'",
 "AOS_AO_MODE_=_'FULL_AO'",
 "TEL_COU_PMA_=_'{}'",
 "TEL_TARG_ADDVELDELTA_=_'{}'",
 "TEL_TARG_EQUINOX_=_'{}'",
 "TEL_TARG_PMA_=_'{}'",
 "TEL_COU_PMD_=_'{}'",
 "TEL_TARG_ADDVELALPHA_=_'{}'",
 "TEL_COU_MAG_=_'{}'",
 "TEL_COU_EQUINOX_=_'{}'",
 "TEL_TARG_ALPHA_=_'{}'",
 "TEL_TARG_PMD_=_'{}E-{}'",
 "TEL_DEL_REF_=_'{}'",
 "TEL_COU_GSTYPE_=_'ADAPT_OPT'",
 "TEL_TARG_DELTA_=_'{}'",
 "TEL_DEL_MODE_=_'AUTO'",
 "TEL_COU_GSSOURCE_=_'SCIENCE'",
 "TEL_DELREF_POS_=_'{}'",
 "TEL_TARG_EPOCH_=_'{}'",
 'Send_command_CHKMOON_{},{},{}_to_process_issifControl_env._wvgvlti',
 'Waiting_(timeout_{})_for_pe

De momento va todo bien, ahora tocaría expandir este cambio al resto del dataset, para ello, desmembremos la función generate_serie (que es la que utilizamos para preparar la data en los otros ficheros)

In [13]:
class NumbersAndTelescopes(pre.Numbers, pre.Telescopes):
    pass

colorize1 = pre.Numbers()
colorize2 = NumbersAndTelescopes()

In [14]:
def generate_serie_SE(obs): 
    #we create the first row for our serie:
    firstrow=[]
    rows=[]
    for a in obs.load_trace(0)["event"]:
        firstrow.append(colorize2.color(a))
        #hacemos el cambio para la primera fila:
        firstrow=list(map(lambda x: x.replace(' ','_'), firstrow))
        
    #and then we create the serie:
    serie= pd.Series([firstrow])
    
    #Finally we repeat this process to create the entire serie
    for i in range(1, len(obs.index)):
    
        for a in obs.load_trace(i)["event"]:
            rows.append(colorize2.color(a))
            #Expandimos el cambio para el resto de la data
            rows=list(map(lambda x: x.replace(' ','_'), rows))
                
        serie=serie.append(pd.Series([rows],index=[i]))
        rows=[]
    
    return serie

Pongan a correr nuestra función y veamos cómo queda la data:

In [15]:
%%time
pionier1_SE=generate_serie_SE(pionier1)

CPU times: total: 1min 24s
Wall time: 1min 24s


In [16]:
#Mostramos los eventos asociados a la trace 0
pionier1_SE[0]

['Started_at_{}_(underlined)',
 "INS_MODE_=_'OBSERV-H'",
 "ISS_IAS_HMAG_=_'{}'",
 "SEQ_OPL_BEAM3_=_'{}'",
 "SEQ_OPL_BEAM4_=_'{}'",
 "INS_DISP_NAME_=_'GRISM'",
 "TEL_COU_ALPHA_=_'{}'",
 "ISS_BASELINE_=_'large_medium'",
 "TEL_COU_EPOCH_=_'{}'",
 "SEQ_DL_PRESET_=_'T'",
 "ISS_VLTITYPE_=_'snapshot'",
 "SEQ_OPL_BEAM1_=_'{}'",
 "SEQ_OPL_BEAM2_=_'{}'",
 "SEQ_PRESET_=_'T'",
 "SEQ_PUPIL_=_'F'",
 "TEL_COU_DELTA_=_'{}'",
 "AOS_AO_MODE_=_'FULL_AO'",
 "TEL_COU_PMA_=_'{}'",
 "TEL_TARG_ADDVELDELTA_=_'{}'",
 "TEL_TARG_EQUINOX_=_'{}'",
 "TEL_TARG_PMA_=_'{}'",
 "TEL_COU_PMD_=_'{}'",
 "TEL_TARG_ADDVELALPHA_=_'{}'",
 "TEL_COU_MAG_=_'{}'",
 "TEL_COU_EQUINOX_=_'{}'",
 "TEL_TARG_ALPHA_=_'{}'",
 "TEL_TARG_PMD_=_'{}E-{}'",
 "TEL_DEL_REF_=_'{}'",
 "TEL_COU_GSTYPE_=_'ADAPT_OPT'",
 "TEL_TARG_DELTA_=_'{}'",
 "TEL_DEL_MODE_=_'AUTO'",
 "TEL_COU_GSSOURCE_=_'SCIENCE'",
 "TEL_DELREF_POS_=_'{}'",
 "TEL_TARG_EPOCH_=_'{}'",
 'Send_command_CHKMOON_{},{},{}_to_process_issifControl_env._wvgvlti',
 'Waiting_(timeout_{})_for_pe

In [17]:
#Mostramos toda la data:
pionier1_SE

0       [Started_at_{}_(underlined), INS_MODE_=_'OBSER...
1       [Started_at_{}_(underlined), DPR_CATG_=_'CALIB...
2       [Started_at_{}_(underlined), PIONIER_gen_cal_k...
3       [PIONIER_gen_cal_dark_--_Dark_calibration_(yel...
4       [PIONIER_acq_--_Acquisition_(yellow), Started_...
                              ...                        
2047    [PIONIER_gen_tec_setup_--_Instrument_setup_(GR...
2048    [Started_at_{}_(underlined), PIONIER_gen_tec_n...
2049    [PIONIER_gen_tec_setup_--_Instrument_setup_(FR...
2050    [Started_at_{}_(underlined), PIONIER_gen_tec_s...
2051    [Started_at_{}_(underlined), DET_NDIT_=_'{}', ...
Length: 2052, dtype: object

In [18]:
#guardamos nuestra data:
file1=open("pionier1_SE","wb")
pickle.dump(pionier1_SE,file1)
file1.close()

In [19]:
#pionier1_SE=pickle.load(open("pionier1_SE","rb"))

Ahora bien, una vez que ya tenemos la información lista para trabajar en el modelo LDA-sklt, corresponde comparar el tamaño de tokens que nos arroja el c_v versus gensim.

In [20]:
#Clase del CountVectorizer()
vect = CountVectorizer(lowercase=False)
#Dejamos todo como str (la info contiene números)
pionier1_SE=pionier1_SE.apply(str)

#Aplicamos el CV
pionier1_SE_vect = vect.fit_transform(pionier1_SE)

In [21]:
pionier1_SE_vect.toarray()
pionier1_SE_vect.shape

(2052, 19447)

In [22]:
#vect.get_feature_names_out(pionier1_SE_vect)

¡Contamos con un total de 19447 tokens únicos!. Veamos ahora Gensim:

In [24]:
#Cargamos nuestro diccionario creado en LDAPionierfilt
dictionary_pionier=models.LdaModel.load("../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary")

INFO:gensim.utils:loading LdaModel object from ../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary
INFO:gensim.utils:Dictionary lifecycle event {'fname': '../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary', 'datetime': '2022-03-02T05:03:16.916049', 'gensim': '4.1.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'loaded'}
INFO:root:dtype was not set in saved Dictionary file ../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary, assuming np.float64
INFO:gensim.utils:loading LdaState object from ../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary.state


In [25]:
print("Number of uniques tokens:",len(dictionary_pionier))

Number of uniques tokens: 33499


Existe una diferencia de tokens alarmante y es momento de parar y pensar un poco qué está pasando. 

Nota: Con el objetivo de mostrar todo el proceso de aprendizaje de este método no planeo borrar los errores sino más bien mostrar cómo van surgiendo y cómo se van resolviendo. 

Lo que está pasando es que si bien haciendo el cambio de "espacio" por " _ " estamos atendiendo a la búsqueda de la premisa event-tokens, no estamos teniendo en cuenta que la puntuación dentro de cada evento es considerado como un separador de palabras para c_v, por lo que necesitamos atender esta situación para continuar con el proceso. Para ello, definiremos una función tokenizadora que irá implementada a c_v la cual hará que únicamente separe las palabras por medio de espacios (y no por puntuación), pero como nosotros ya nos deshicimos de estos no habría problema. Veamos un ejemplo:

In [26]:
#Como lo estábamos trabajando antes:
corpus2 = [
    'This_is_the_first_document.',
    'This_is_the_first_document.',
    'This_document_is_the_second_document.',
    'And_this_is_the_third_one.',]

#Clase del CountVectorizer()
ve = CountVectorizer()
#Aplicamos el CV
corpus2_ve = ve.fit_transform(corpus2)
#Mostramos los nombres:
ve.get_feature_names_out()

array(['and_this_is_the_third_one',
       'this_document_is_the_second_document',
       'this_is_the_first_document'], dtype=object)

Podemos notar cómo los espacios permiten que se respete la premisa event-tokens, pero agreguémosle caracteres especiales y puntuación:

In [27]:
corpus2_p = [
    'This_is_the_.first_document.',
    'This_is_the_{first_document.',
    'This_docum.ent_is_the_second_document.',
    'And_this_is_the_third_one.',]
corpus2_p_ve = ve.fit_transform(corpus2_p)
ve.get_feature_names_out(corpus2_p_ve)

array(['and_this_is_the_third_one', 'ent_is_the_second_document',
       'first_document', 'this_docum', 'this_is_the_'], dtype=object)

Vemos claramente como la frase 'This_document_is_the_second_document.' la divide en 'this_docum' y 'ent_is_the_second_document.', que es justamente lo que queremos evitar. Implementemos ahora esta función tokenizadora que mencionamos anteriormente:

In [28]:
import re
def my_tokenizer(text):
    # split based on whitespace
    return re.split("\\s+",text)

Veamos qué tal funciona:

In [29]:
cv = CountVectorizer(tokenizer=my_tokenizer)
corpus2_pnew=cv.fit_transform(corpus2_p)
print(cv.vocabulary_)

{'this_is_the_.first_document.': 2, 'this_is_the_{first_document.': 3, 'this_docum.ent_is_the_second_document.': 1, 'and_this_is_the_third_one.': 0}


 ¡Funciona de maravilla!, contamos con un diccionario de 4 elementos aún teniendo puntuación nuestras oraciones, y manteniendo los caracteres especiales (al igual que el gensim). Entonces:
 
    --Reemplazamos los espacios por " _ " para evitar la separación de palabra por palabra: *check*
    --Evitamos la separación de palabra por palabra a través de la puntuación: *check*
    --Conservamos los caracteres especiales: *check*
    

Apliquemos todo lo anterior a nuestro dataset:

In [30]:
#Clase del CountVectorizer()
vecto = CountVectorizer(tokenizer=my_tokenizer,lowercase=False)
#Dejamos todo como str (la info contiene números)
pionier1_SE=pionier1_SE.apply(str)

#Aplicamos el CV
pionier1_SE_vecto = vecto.fit_transform(pionier1_SE)

In [31]:
pionier1_SE_vecto.toarray()
pionier1_SE_vecto.shape

(2052, 33541)

Mucho mejor, pasamos de 19447 a 33541 tokens. Aún no coincidimos con el diccionario de gensim debido a que LDA-gensim fue ejecutado con información de entrenamiento equivalente al 80% del dataset total. Importemos el modelo LDA-gensim con el dataset total:

In [32]:
dictionary_pionier_DT=models.LdaModel.load("../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary")

INFO:gensim.utils:loading LdaModel object from ../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary
INFO:gensim.utils:Dictionary lifecycle event {'fname': '../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary', 'datetime': '2022-03-02T05:03:55.769394', 'gensim': '4.1.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'loaded'}
INFO:root:dtype was not set in saved Dictionary file ../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary, assuming np.float64
INFO:gensim.utils:loading LdaState object from ../../LDAmodel-gensim/Pionnier/Pioniermodelsave/pionierDT.dictionary.state


In [33]:
print("Number of uniques tokens:",len(dictionary_pionier_DT))

Number of uniques tokens: 33499


¡Tenemos una diferencia de 42 tokens!. ¿A qué se debe esto?, para ser sincero, no cuento con la respuesta, me imagino que puede ser porque alguna oración terminase con un espacio y el modelo le implementase un " _ " al final:

In [34]:
Frase=['Frase de prueba con espacio al final ','Frase de prueba con espacio al final']
Frase_d = list(map(lambda x: x.replace(' ','_'), Frase))

In [35]:
Frase_d

['Frase_de_prueba_con_espacio_al_final_',
 'Frase_de_prueba_con_espacio_al_final']

La herramienta .strip() (elimina espacios del comienzo y término para cada elemento de una lista) puede ayudarnos en este caso, veamos:   

In [36]:
Frase_dd=list(map(lambda x: x.strip(), Frase))
Frase_dd

['Frase de prueba con espacio al final',
 'Frase de prueba con espacio al final']

Agreguemos entonces esta herramienta a nuestra función:

In [37]:
def generate_serie_SES(obs): 
    #we create the first row for our serie:
    firstrow=[]
    rows=[]
    for a in obs.load_trace(0)["event"]:
        firstrow.append(colorize2.color(a))
        #borramos el espacio del comienzo y final
        firstrow=list(map(lambda x: x.strip(), firstrow))
        #hacemos el cambio para la primera fila:
        firstrow=list(map(lambda x: x.replace(' ','_'), firstrow))
        
    #and then we create the serie:
    serie= pd.Series([firstrow])
    
    #Finally we repeat this process to create the entire serie
    for i in range(1, len(obs.index)):
    
        for a in obs.load_trace(i)["event"]:
            rows.append(colorize2.color(a))
            #borramos el espacio del comienzo y final de los demás
            rows=list(map(lambda x: x.strip(), rows))
            #Expandimos el cambio para el resto de la data
            rows=list(map(lambda x: x.replace(' ','_'), rows))
                
        serie=serie.append(pd.Series([rows],index=[i]))
        rows=[]
    
    return serie

In [38]:
%%time
pionier1_SES=generate_serie_SES(pionier1)

CPU times: total: 1min 33s
Wall time: 1min 33s


In [41]:
#guardamos nuestra data:
file2=open("pionier1_SES","wb")
pickle.dump(pionier1_SES,file2)
file2.close()

In [ ]:
##Checkpoint:
#pionier1_SES=pickle.load(open("pionier1_SES","rb"))

In [39]:
#Clase del CountVectorizer()
vecto = CountVectorizer(tokenizer=my_tokenizer,lowercase=False)
#Dejamos todo como str (la info contiene números)
pionier1_SES=pionier1_SES.apply(str)

#Aplicamos el CV
pionier1_SES_vecto = vecto.fit_transform(pionier1_SES)

In [40]:
pionier1_SES_vecto.toarray()
pionier1_SES_vecto.shape

(2052, 33541)

Sigue esta diferencia de 42 tokens, desconozco la razón, pero lo más probable es que sea producto de algún token de frecuencia mínima por lo que no afecta en el desarrollo del modelo.

**Apliquemos esto mismo a las otros dataset:**

**Matisse**:

In [240]:
Matisse1=TplMatisse(config=config.ds_medium)
# Some rows are small (<5 sec) or unbounded (>30m), this criteria will restrict to only the good ones
Matisse1.index = Matisse1.index[Matisse1.index["SECONDS"] > 5]
Matisse1.index = Matisse1.index[Matisse1.index["SECONDS"] < 1800]
Matisse1.index

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,estimated (s),real (s),diff (s),SECONDS
0,2020-12-03 22:28:21.278000+00:00,2020-12-03 22:54:02.577000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_ali,False,False,1020,2541,921,1541.0
1,2020-12-03 22:59:52.185000+00:00,2020-12-03 23:11:34.211000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_ali,False,False,1020,1142,82,702.0
2,2020-12-04 18:12:58.130000+00:00,2020-12-04 18:33:35.254000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_ali,False,False,900,2037,697,1237.0
3,2020-12-04 18:34:19.053000+00:00,2020-12-04 18:34:57.750000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_ali,False,False,900,38,502,38.0
5,2020-12-04 18:37:24.312000+00:00,2020-12-04 18:41:29.024000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_opd,False,False,320,405,45,244.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1311,2021-01-01 20:15:23.694000+00:00,2021-01-01 20:26:46.564000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_ali,False,False,1020,1123,63,682.0
1312,2021-01-01 20:30:27.665000+00:00,2021-01-01 20:33:40.689000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_opd,False,False,320,313,7,193.0
1313,2021-01-01 20:33:40.733000+00:00,2021-01-01 20:34:19.906000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_opd,False,False,130,39,51,39.0
1314,2021-01-01 20:34:19.936000+00:00,2021-01-01 20:37:26.449000+00:00,False,MATISSE,bob_ins,MATISSE_gen_tec_opd,False,False,320,307,13,186.0


In [242]:
%%time
Matisse1_SES=generate_serie_SES(Matisse1)

CPU times: total: 8min 59s
Wall time: 9min 9s


In [243]:
#Clase del CountVectorizer()
vecto = CountVectorizer(tokenizer=my_tokenizer,lowercase=False)
#Dejamos todo como str (la info contiene números)
Matisse1_SES=Matisse1_SES.apply(str)

#Aplicamos el CV
Matisse1_SES_vecto = vecto.fit_transform(Matisse1_SES)

In [244]:
Matisse1_SES_vecto.toarray()
Matisse1_SES_vecto.shape
#79900 tokens

(1180, 79900)

In [246]:
#importamos el modelo en gensim
dictionary_matisse_DT=models.LdaModel.load("../Matisse/Matissemodelsave/matisseDT.dictionary")

INFO:gensim.utils:loading LdaModel object from ../Matisse/Matissemodelsave/matisseDT.dictionary
INFO:gensim.utils:Dictionary lifecycle event {'fname': '../Matisse/Matissemodelsave/matisseDT.dictionary', 'datetime': '2022-03-02T02:43:07.881528', 'gensim': '4.1.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'loaded'}
INFO:root:dtype was not set in saved Dictionary file ../Matisse/Matissemodelsave/matisseDT.dictionary, assuming np.float64
INFO:gensim.utils:loading LdaState object from ../Matisse/Matissemodelsave/matisseDT.dictionary.state


In [247]:
print("Number of uniques tokens:",len(dictionary_matisse_DT))

Number of uniques tokens: 79846


**Gravity**:

In [249]:
gravity1=TplGravity(config=config.ds_medium)
# Some rows are small (<5 sec) or unbounded (>30m), this criteria will restrict to only the good ones
gravity1.index = gravity1.index[gravity1.index["SECONDS"] > 5]
gravity1.index = gravity1.index[gravity1.index["SECONDS"] < 1800]
gravity1.index

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,estimated (s),real (s),diff (s),SECONDS
0,2020-12-01 00:16:04.547000+00:00,2020-12-01 00:19:11.750000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,227,307,40,187.0
1,2020-12-01 00:19:11.804000+00:00,2020-12-01 00:23:49.341000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,438,22,277.0
2,2020-12-01 00:23:49.411000+00:00,2020-12-01 00:25:34.908000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,145,7,105.0
3,2020-12-01 00:25:34.967000+00:00,2020-12-01 00:29:33.402000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,359,61,238.0
4,2020-12-01 00:29:33.453000+00:00,2020-12-01 00:31:18.444000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,145,7,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1242,2021-01-01 19:55:07.919000+00:00,2021-01-01 19:59:19.367000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_AcqFibA,False,False,400,412,12,251.0
1243,2021-01-01 19:59:19.458000+00:00,2021-01-01 20:10:17.247000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_AcqFibB,False,False,1125,1058,27,657.0
1245,2021-01-01 20:10:20.661000+00:00,2021-01-01 20:11:58.709000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_PTref,False,False,135,138,3,98.0
1246,2021-01-01 20:11:58.798000+00:00,2021-01-01 20:13:26.902000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_WFSref,False,False,123,128,5,88.0


In [250]:
%%time
gravity1_SES=generate_serie_SES(gravity1)

CPU times: total: 19min 6s
Wall time: 19min 20s


In [251]:
#Clase del CountVectorizer()
vecto = CountVectorizer(tokenizer=my_tokenizer,lowercase=False)
#Dejamos todo como str (la info contiene números)
gravity1_SES=gravity1_SES.apply(str)

#Aplicamos el CV
gravity1_SES_vecto = vecto.fit_transform(gravity1_SES)

In [252]:
gravity1_SES_vecto.toarray()
gravity1_SES_vecto.shape
#79900 tokens

(992, 12940)

In [253]:
#importamos el modelo en gensim
dictionary_gravity_DT=models.LdaModel.load("../Gravity/Gravitymodelsave/gravityDT.dictionary")

INFO:gensim.utils:loading LdaModel object from ../Gravity/Gravitymodelsave/gravityDT.dictionary
INFO:gensim.utils:Dictionary lifecycle event {'fname': '../Gravity/Gravitymodelsave/gravityDT.dictionary', 'datetime': '2022-03-02T03:40:41.194702', 'gensim': '4.1.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'loaded'}
INFO:root:dtype was not set in saved Dictionary file ../Gravity/Gravitymodelsave/gravityDT.dictionary, assuming np.float64
INFO:gensim.utils:loading LdaState object from ../Gravity/Gravitymodelsave/gravityDT.dictionary.state


In [255]:
print("Number of uniques tokens:",len(dictionary_gravity_DT))

Number of uniques tokens: 12903


#### Como ya disponemos de la teoría, veamos en la práctica